In [170]:
import os
import csv
import time
import datetime
import urllib
import re
import math
import sys
import io
import random
import pymysql.cursors
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
style.use("ggplot")
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score

#### Set connection to mySQL server

In [ ]:
connection = pymysql.connect(host='10.0.106.71', 
                             user='roylee', 
                             password='LARCuser4848', 
                             db='roylee',
                             cursorclass=pymysql.cursors.DictCursor)


#### Loading the base users from file
- i_file: input file

In [54]:
i_file = 'data/SOGH_users.csv'
so_gh_users = {}
gh_so_users = {}

with open(i_file) as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        ghid = str(row[0])
        soid = str(row[1])
        so_gh_users[soid] = ghid
        gh_so_users[ghid] = soid

#### Get watch activity data from mySQL
- i_start_date: start date for the range of data retrieved
- i_end_date: end date for the range of data retrieved
- o_groudtruth_file: user-watch activity file
- o_repo_file: watched repository with the description tags

In [ ]:
i_start_date = ''
i_end_date = ''
o_groundtruth_file = ''
o_repo_file= ''

try:
    with open(o_groundtruth_file, 'a+') as fout:
        with connection.cursor() as cursor:
            count = 0
            base_watch_repos = {}
            for key, value in gh_so_users.items():
                sql = "SELECT ownerid, repoid, watcherid, tags FROM sb_ghwatcher WHERE watcherid='"+key+ 
                "' AND create_at>='"+i_start_date+
                "' AND create_at<'"+i_end_date+"'"
                
                cursor.execute(sql)
                result = cursor.fetchall()
                for line in result:
                    ownerid = line['ownerid']
                    repoid = line['repoid']
                    watcherid = line['watcherid']
                    tags = line['tags'].strip()
                    if tags != 'null':
                        fout.writelines(watcherid +','+ repoid + '\n')
                        base_watch_repos[repoid] = tags
                count += 1
                print(count)
    with open(o_repo_file, 'a+') as fout:
        for key, value in base_watch_repos.items():
            fout.writelines( key +','+ value + '\n')
finally:
    connection.close()

#### Get fork activity data from mySQL
- i_start_date: start date for the range of data retrieved
- i_end_date: end date for the range of data retrieved
- o_groudtruth_file: user-fork activity file
- o_repo_file: forked repository with the description tags

In [ ]:
i_start_date = ''
i_end_date = ''
o_groundtruth_file = ''
o_repo_file= ''

try:
    with open(o_groundtruth_file, 'a+') as fout:
        with connection.cursor() as cursor:
            count = 0
            base_fork_repos = {}
            for key, value in gh_so_users.items():
                sql = "SELECT ownerid, repoid, tags FROM sb_ghfork WHERE ownerid='"+key+
                "' AND create_at>='"+i_start_date+
                "' AND create_at<'"+i_end_date+"'"
                
                cursor.execute(sql)
                result = cursor.fetchall()
                for line in result:
                    ownerid = line['ownerid']
                    repoid = line['repoid']
                    tags = line['tags'].strip()
                    if tags != 'null':
                        fout.writelines(ownerid +','+ repoid + '\n')
                        base_fork_repos[repoid] = tags
                count += 1
                print(count)
    with open(o_repo_file, 'a+') as fout:
        for key, value in base_fork_repos.items():
            fout.writelines(key +','+ value + '\n')
finally:
    connection.close()

#### Get answer activity data from mySQL
- i_start_date: start date for the range of data retrieved
- i_end_date: end date for the range of data retrieved
- o_groudtruth_file: user-answer activity file
- o_qn_file: answered question with the description tags

In [ ]:
i_start_date = ''
i_end_date = ''
o_groundtruth_file = ''
o_qn_file= ''

try:
    with open(o_groundtruth_file, 'a+') as fout:
        with connection.cursor() as cursor:
            count = 0
            base_answer_questions = {}
            for key, value in gh_so_users.items():
                sql = "SELECT ownerid, parentid, tags FROM sb_soanswer WHERE ownerid='"+value+
                "' AND create_at>='"+i_start_date+
                "' AND create_at<'"+i_end_date+"'"
                
                cursor.execute(sql)
                result = cursor.fetchall()
                for line in result:
                    ownerid = line['ownerid']
                    qnid = line['parentid']
                    tags = line['tags'].strip()
                    if tags != 'null':
                        fout.writelines(ownerid +','+ qnid + '\n')
                        base_answer_questions[qnid] = tags
                count += 1
                print(count)
    with open(o_qn_file, 'a+') as fout:
        for key, value in base_answer_questions.items():
            fout.writelines(key +','+ value + '\n')
finally:
    connection.close()

#### Get favorite activity data from mySQL
- i_start_date: start date for the range of data retrieved
- i_end_date: end date for the range of data retrieved
- o_groudtruth_file: user-favorite activity file
- o_qn_file: favorited question with the description tags

In [ ]:
i_start_date = ''
i_end_date = ''
o_groundtruth_file = ''
o_qn_file= ''

try:
    with open(o_groundtruth_file, 'a+') as fout:
        with connection.cursor() as cursor:
            count = 0
            base_favorite_questions = {}
            for key, value in gh_so_users.items():
                sql = "SELECT ownerid, parentid, tags FROM sb_sofavorite WHERE ownerid='"+value+
                "' AND create_at>='"+i_start_date+
                "' AND create_at<'"+i_end_date+"'"
                
                cursor.execute(sql)
                result = cursor.fetchall()
                for line in result:
                    ownerid = line['ownerid']
                    qnid = line['parentid']
                    tags = line['tags'].strip()
                    if tags != 'null':
                        fout.writelines(ownerid +','+ qnid + '\n')
                        base_favorite_questions[qnid] = tags
                count += 1
                print(count)
    with open(o_qn_file, 'a+') as fout:
        for key, value in base_favorite_questions.items():
            fout.writelines(key +','+ value + '\n')
finally:
    connection.close()

#### Generate GitHub training/testing data
This generate the positive and negative activity instances for each user.
- i_groundtruth_file: user-activity file
- i_repo_file: activity repository with the description tags
- o_data_file: output data file
- n: number of negative instances = n * number of positive instances 

In [ ]:
i_groundtruth_file = '' 
i_repo_file = '' 
o_data_file = ''
n = 5

user_repo_pos_count = {}
user_repo_neg_count = {}
pos_data = []
neg_data = []
repos = []

for key, value in gh_so_users.items():
    user_repo_pos_count[key] = 0

with open(i_repo_file) as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        repoid = str(row[0])
        repos.append(repoid)

with open(i_groundtruth_file) as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        ghid = str(row[0])
        repoid = str(row[1])
        user_repo_pos_count[ghid] = user_repo_pos_count[ghid] + 1
        pos_data.append(ghid+','+repoid)

for key,value in user_repo_pos_count.items():
    user_repo_neg_count[key] = value * n

count = 0
for key,value in user_repo_neg_count.items():
    ghid = key
    while value != 0:
        candidate = random.choice(repos)
        neg_choice = ghid+','+candidate
        if neg_choice not in pos_data:
            neg_data.append(neg_choice)
            value -= 1
    count += 1
    print(count)
    
with open(o_data_file, 'a+') as fout:
    for item in pos_data:
        fout.writelines(item +',1'+'\n')
    for item in neg_data:
        fout.writelines(item +',-1'+'\n')

#### Generate Stack Overflow training/testing data
This generate the positive and negative activity instances for each user.
- i_groundtruth_file: user-activity file
- i_qn_file: activity question with the description tags
- o_data_file: output data file
- n: number of negative instances = n * number of positive instances 

In [ ]:
i_groundtruth_file = '' 
i_qn_file = '' 
o_data_file = ''
n = 5

user_qn_pos_count = {}
user_qn_neg_count = {}
pos_data = []
neg_data = []
qns = []

for key, value in gh_so_users.items():
    user_qn_pos_count[value] = 0


with open(i_qn_file) as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        qnid = str(row[0])
        qns.append(qnid)

with open(i_groundtruth_file) as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        soid = str(row[0])
        qnid = str(row[1])
        user_qn_pos_count[soid] = user_qn_pos_count[soid] + 1
        pos_data.append(soid+','+qnid)

for key,value in user_qn_pos_count.items():
    user_qn_neg_count[key] = value * 5

count = 0
for key,value in user_qn_neg_count.items():
    soid = key
    while value != 0:
        candidate = random.choice(qns)
        neg_choice = soid+','+candidate
        if neg_choice not in pos_data:
            neg_data.append(neg_choice)
            value -= 1
    count += 1
    print(count)
    
with open(o_data_file, 'a+') as fout:
    for item in pos_data:
        fout.writelines(item +',1'+'\n')
    for item in neg_data:
        fout.writelines(item +',-1'+'\n')

#### Get the user activity distribution
- i_fork_file: input user-fork activity file
- i_watch_file: input user-watch activity file
- i_answer_file: input user-answer activity file
- i_favorite_file: input user-favorite activity file
- o_dist_file: output user distribution file

In [51]:
i_fork_file = 'toy_data/user_fork_repository.csv'
i_watch_file = 'toy_data/user_watch_repository.csv'
i_answer_file = 'toy_data/user_answer_question.csv'
i_favorite_file = 'toy_data/user_favorite_question.csv'
o_dist_file = 'toy_data/user_activity_distribution.csv'

so_dist = {} #soid,answer_count,fav_count
gh_dist = {} #ghid,fork_count,watch_count

for key, value in gh_so_users.items():
    so_dist[value] = [0,0]
    gh_dist[key] = [0,0]
    
with open(i_fork_file) as f:
    reader = csv.reader(f,delimiter=',')
    for row in reader:
        uid = str(row[0])
        gh_dist[uid][0] += 1

with open(i_watch_file) as f:
    reader = csv.reader(f,delimiter=',')
    for row in reader:
        uid = str(row[0])
        gh_dist[uid][1] += 1

with open(i_answer_file) as f:
    reader = csv.reader(f,delimiter=',')
    for row in reader:
        uid = str(row[0])
        so_dist[uid][0] += 1

with open(i_favorite_file) as f:
    reader = csv.reader(f,delimiter=',')
    for row in reader:
        uid = str(row[0])
        so_dist[uid][1] += 1

with open(o_dist_file, 'a+') as fout:
    for key, value in gh_so_users.items():
        fout.writelines(str(key) +','+ 
            str(value) +','+ 
            str(gh_dist[key][0]) +','+
            str(gh_dist[key][1]) +','+
            str(so_dist[value][0]) +','+
            str(so_dist[value][1]) + '\n')

#### Describe the user-activity distribution
- i_user_dist_file: the user distribution file

In [55]:
i_user_dist_file = 'data/user_activity_distribution.csv'

df = pd.read_csv(i_user_dist_file, header=None)
df.columns = ['gh_id','so_id','fork_count','watch_count','answer_count','favorite_count']
df.describe()

,gh_id,so_id,fork_count,watch_count,answer_count,favorite_count
count,4.623900e+04,4.623900e+04,46239.000000,46239.000000,46239.000000,46239.000000
mean,7.037569e+05,8.805124e+05,3.702199,18.190164,11.762819,4.359458
std,6.940880e+05,7.345248e+05,8.399300,69.868085,67.255959,25.402997
min,2.000000e+00,1.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,1.443965e+05,2.371610e+05,0.000000,0.000000,0.000000,0.000000
50%,4.297700e+05,6.978560e+05,1.000000,2.000000,1.000000,0.000000
75%,1.122494e+06,1.393410e+06,4.000000,11.000000,5.000000,1.000000
max,7.579459e+06,2.736012e+06,448.000000,4939.000000,3959.000000,2730.000000


#### Reformat data into new format
- i_user_references: reference for users's GitHub and Stack Overflow user ids
- i_user_answer: user's answer activities (old format)
- i_user_favorite = user's favorite activities (old format)
- i_user_fork = user's fork activities (old format)
- i_user_watch = user's watch activities (old format)
- i_answer_tag = tags for answered questions
- i_favorite_tag = tags for favorited questions
- i_fork_tag = tags for forked repositories
- i_watch_tag = tags for watched repositories
- o_user_answer = output file for user's answer activities
- o_user_favorite = output file for user's favorite activities
- o_user_fork = output file for user's fork activities
- o_user_watch = output file for user's watch activities

In [67]:
i_user_references = 'toy_data/new_format/user_references.csv'
i_user_answer = 'toy_data/user_answer_question.csv'
i_user_favorite = 'toy_data/user_favorite_question.csv'
i_user_fork = 'toy_data/user_fork_repository.csv'
i_user_watch = 'toy_data/user_watch_repository.csv'
i_answer_tag = 'toy_data/question_answer_tag.csv'
i_favorite_tag = 'toy_data/question_favorite_tag.csv'
i_fork_tag = 'toy_data/repository_fork_tag.csv'
i_watch_tag = 'toy_data/repository_watch_tag.csv'
o_user_answer = 'toy_data/new_format/user_answer_activities.csv'
o_user_favorite = 'toy_data/new_format/user_favorite_activities.csv'
o_user_fork = 'toy_data/new_format/user_fork_activities.csv'
o_user_watch = 'toy_data/new_format/user_watch_activities.csv'

gh_uid = {}
so_uid = {}
with open(i_user_references, encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        uid = int(row[0])
        ghid = str(row[1])
        soid = str(row[2])
        gh_uid[ghid] = uid
        so_uid[soid] = uid

qn_tags = {}
with open(i_answer_tag, encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        qid = str(row[0])
        tags = str(row[1])
        tags = tags.replace(' ; ',' ')
        qn_tags[qid] = tags
        
with open(i_favorite_tag, encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        qid = str(row[0])
        tags = str(row[1])
        tags = tags.replace(' ; ',' ')
        qn_tags[qid] = tags
        
repo_tags = {}
with open(i_fork_tag, encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        rid = str(row[0])
        tags = str(row[1])
        tags = tags.replace(' ; ',' ')
        repo_tags[rid] = tags
        
with open(i_watch_tag, encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        rid = str(row[0])
        tags = str(row[1])
        tags = tags.replace(' ; ',' ')
        repo_tags[rid] = tags
        
with open(o_user_answer, 'a+') as fout:
    with open(i_user_answer, encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=',')
        for row in reader:
            soid = str(row[0])
            qid = str(row[1])
            uid = so_uid[soid]
            tags = qn_tags[qid]
            fout.writelines(str(uid) +','+ str(qid) +','+ str(tags) + '\n')
        
with open(o_user_favorite, 'a+') as fout:
    with open(i_user_favorite, encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=',')
        for row in reader:
            soid = str(row[0])
            qid = str(row[1])
            uid = so_uid[soid]
            tags = qn_tags[qid]
            fout.writelines(str(uid) +','+ str(qid) +','+ str(tags) + '\n')

with open(o_user_fork, 'a+') as fout:
    with open(i_user_fork, encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=',')
        for row in reader:
            ghid = str(row[0])
            rid = str(row[1])
            uid = gh_uid[ghid]
            tags = repo_tags[rid]
            fout.writelines(str(uid) +','+ str(rid) +','+ str(tags) + '\n')

with open(o_user_watch, 'a+') as fout:
    with open(i_user_watch, encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=',')
        for row in reader:
            ghid = str(row[0])
            rid = str(row[1])
            uid = gh_uid[ghid]
            tags = repo_tags[rid]
            fout.writelines(str(uid) +','+ str(rid) +','+ str(tags) + '\n')

#### Genenerate Training and Test set
- i_users: user file
- i_activities: user activity file
- o_training_activities: output training file for the activity
- o_testing_activities: output test file for the activity
- percentage_train: percentage of user activities used for traning
- nNegative: Number of negative instances, which is the n times of positive instance

In [88]:
i_users = 'toy_data/new_format/users.csv'
i_activities = 'toy_data/new_format/user_favorite_activities.csv'
o_training_activities = 'toy_data/new_format/training/user_favorite_training.csv'
o_testing_activities = 'toy_data/new_format/test/user_favorite_testing.csv'
percentage_train = 0.8
nNegative = 5

user_acitivity_counts = {}
user_activities = {}
user_non_activities = {}
with open(i_users, encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        uid = int(row[0])       
        user_acitivity_counts[uid] = 0
        user_activities[uid] = []
        user_non_activities[uid] = []

all_activities = []
activity_tags = {}
with open(i_activities, encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        uid = int(row[0])
        rid = str(row[1])
        tags = str(row[2])
        user_acitivity_counts[uid] += 1
        user_activities[uid].append(rid)
        all_activities.append(rid)
        activity_tags[rid] = tags

for key,value in user_non_activities.items():
    user_non_activities[key] = list(set(all_activities)-set(user_activities[key]))
       
user_training_count_pos = {}
user_training_count_neg = {}
user_test_count_neg = {}
for key, value in user_acitivity_counts.items():
    user_training_count_pos[key] = int(math.ceil(user_acitivity_counts[key] * percentage_train))
    
    #if it is bigger than all the possible non-links
    if (value * nNegative) > len(user_non_activities[key]):
        user_training_count_neg[key] = int(math.ceil(len(user_non_activities[key]) * percentage_train))
        user_test_count_neg[key] = int(math.floor(len(user_non_activities[key]) * (1-percentage_train)))
    else:
        user_training_count_neg[key] = int(math.ceil(user_acitivity_counts[key] * percentage_train) * nNegative)
        user_test_count_neg[key] = int(math.floor(user_acitivity_counts[key] * (1-percentage_train)) * nNegative)

#Generate positive training and test instances
with open(o_training_activities, 'a+') as ftrain:
    with open(o_testing_activities, 'a+') as ftest:
        with open(i_activities, encoding='utf-8') as f:
            reader = csv.reader(f, delimiter=',')
            for row in reader:
                uid = int(row[0])
                rid = str(row[1])
                tags = str(row[2])
                if user_training_count_pos[uid] > 0:
                    ftrain.writelines(str(uid) +','+ str(rid) +','+ str(tags) + ',1\n')
                    user_training_count_pos[uid] -= 1
                else:
                    ftest.writelines(str(uid) +','+ str(rid) +','+ str(tags) + ',1\n')

#Generate negative training instances
with open(o_training_activities, 'a+') as ftrain:
    for key, value in user_training_count_neg.items():
        random.shuffle(user_non_activities[key])
        for i in range(0,value):
            rid = user_non_activities[key].pop()
            tags = activity_tags[rid]
            ftrain.writelines(str(key) +','+ str(rid) +','+ str(tags) +',0 \n')
            
#Generate negative test instances
with open(o_testing_activities, 'a+') as ftest:
    for key, value in user_test_count_neg.items():
        random.shuffle(user_non_activities[key])
        for i in range(0,value):
            rid = user_non_activities[key].pop()
            tags = activity_tags[rid]
            ftest.writelines(str(key) +','+ str(rid) +','+ str(tags) +',0 \n')

## Activity Similarity Feature: Similary between query item and user's activities
Given a query item $q$ (i.e., a Stack Overflow question or GitHub repository) and a user $u$, we compute averange similarity between $q$ and all items where $u$ has perform an activity $a$ on. For example, given an item question $q_1$, we compute the average similarity $q_1$ and all other questions where beween user $u_1$ has answered. I.e., in this case, the activity $a$ will be the *answer* activity. The similarity function is given as below: 

$Sim(u,q,a) = \frac{|\{i\in I_{(u,a)}|i_{tags}\in q_{tags}\}|}{|I_{(u,a)}|}$

Where, $<u,q,a>$ is a triplet of a user $u$, query item $q$ and specific activity $a$. We say that a query item $q$ is similar to a user $u$'s $a$ activities, when many items, which $u$ performed $a$ on, shares similar tags with $q$. The above similarity function captures this intuition. The numerator computes the number of items where $u$ perform $a$ and the item shares at least 1 tag with the query item $q$. The denominator computes the total number of items where $u$ perform $a$

Notations
- $u$: User
- $a$: Activity. Different activities performed by user. E.g. answer, favorite, fork and watch.
- $i$: Item, i.e., Stack Overflow question or GitHub repository
- $q$: Query item
- $I_{(u,a)}$: Items where user $u$ perform activity $a$ on. E.g. questions which are answered by a user. 
- $i_{tags}$: Tags for the item. E.g. *Java*, *iOS*, etc. 
- $q_{tags}$: Tags for query item.

For function UserActivitySim():
- q: query item id (str)
- I_ua: ids of items where user has perform an activity (list)
- I_tags: key is item id and value is tags (dict)

In [149]:
def UserActivitySim(q, I_ua, I_tags):
    if q in I_ua:
        I_ua.remove(q)
    if len(I_ua)==0:
        return 0
    q_tags = set(I_tags[q].split(' '))
    numerator = 0
    for i in I_ua:
        i_tags = set(I_tags[i].split(' '))
        overlap = i_tags.intersection(q_tags)
        if len(overlap) >0:
            numerator += 1
    return numerator/len(I_ua)

## Evaluation
We use [Mean Average Precision (MAP)](https://medium.com/@jonathan_hui/map-mean-average-precision-for-object-detection-45c121a31173) to evaluate the activity prediciton tasks. I.e., we take total average precision (AP) of each user divided by the total number of users. 

MAP function:
- groundtruth: a dictionary where key is user id and value is the test labels for all the positive/negative instance for this use
- pred: a dictionary where key is the user id and value is the predicted probabilities of the label being posititive (i.e,'1')

In [186]:
def MAP(groundtruth,pred):
    result = 0
    for key, value in groundtruth.items():
        y_truth = value
        y_pred = pred[key]
        score= average_precision_score(y_truth,y_pred)
        result +=score
    return result/len(groundtruth)

### Answer Activity Prediction 
#### Setup 1: Using only Answer Similarity Feature
- Compute the similarity scores for answer training set

In [150]:
i_users = 'toy_data/new_format/users.csv'
i_training_activities = 'toy_data/new_format/training/user_answer_training.csv'
i_testing_activities = 'toy_data/new_format/test/user_answer_testing.csv'

#Load answer training set
ans_u_ids_train = []
ans_q_ids_train = []
ans_q_label_train = []
ans_user_items_train = {} 
ans_item_tags_train = {}
with open(i_training_activities, encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        uid = int(row[0])
        rid = str(row[1])
        tags = str(row[2])
        label = int(row[3])
        ans_u_ids_train.append(uid)
        ans_q_ids_train.append(rid)
        ans_q_label_train.append(label)
        if label==1:
            if uid in ans_user_items_train:
                ans_user_items_train[uid].append(rid)
            else:
                ans_user_items_train[uid] = [rid]
        ans_item_tags_train[rid] = tags

#Compute score for answer training set
ans_scores_train = []
for i in range(len(ans_u_ids_train)):
    q = ans_q_ids_train[i]
    uid = ans_u_ids_train[i]
    I_ua = ans_user_items_train[uid].copy()
    I_tags = ans_item_tags_train
    score = UserActivitySim(q, I_ua, I_tags)
    ans_scores_train.append(score)
    
#Load answer test set
ans_u_ids_test = []
ans_q_ids_test = []
ans_q_label_test = []
ans_user_items_test = {} 
ans_item_tags_test = {}
with open(i_testing_activities, encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        uid = int(row[0])
        rid = str(row[1])
        tags = str(row[2])
        label = int(row[3])
        ans_u_ids_test.append(uid)
        ans_q_ids_test.append(rid)
        ans_q_label_test.append(label)
        if label==1:
            if uid in ans_user_items_test:
                ans_user_items_test[uid].append(rid)
            else:
                ans_user_items_test[uid] = [rid]
        ans_item_tags_test[rid] = tags

#Compute score for answer test set
ans_scores_test = []
for i in range(len(ans_u_ids_test)):
    q = ans_q_ids_test[i]
    uid = ans_u_ids_test[i]
    I_ua = ans_user_items_test[uid].copy()
    I_tags = ans_item_tags_test

    score = UserActivitySim(q, I_ua, I_tags)
    ans_scores_test.append(score)

In [190]:
#Training
x_train_list = []
for i in range(len(ans_scores_train)):
    x_train_list.append([int(ans_u_ids_train[i]),int(ans_scores_train[i])])
x_train = np.array(x_train_list)
y_train = ans_q_label_train.copy()
clf = svm.SVC(kernel='linear',probability=True)
#clf = svm.SVC(kernel='linear')
clf.fit(x_train_list,y_train)

#Testing
x_test_list = []
for i in range(len(ans_scores_test)):
    x_test_list.append([int(ans_u_ids_test[i]),int(ans_scores_test[i])])
x_test = np.array(x_test_list)
y_test = ans_q_label_test.copy()
pred = clf.predict(x_test)

In [191]:
#evaluation
user_level_truth = {}
user_level_pred = {}
for i in range(len(pred)):
    uid = ans_u_ids_test[i]
    if uid not in user_level_pred:
        user_level_pred[uid] = [pred[i][0]]
    else:
        user_level_pred[uid].append(pred[i][1])

for i in range(len(ans_q_label_test)):
    uid = ans_u_ids_test[i]
    if uid not in user_level_truth:
        user_level_truth[uid] = [ans_q_label_test[i]]
    else:
        user_level_truth[uid].append(ans_q_label_test[i])

result = MAP(user_level_truth,user_level_pred)
print(result)

IndexError: invalid index to scalar variable.